In [1]:
# Imports

from keras import Sequential
from keras.layers import LSTM, Dense, Dropout, Flatten, Activation, Reshape
from keras.losses import categorical_crossentropy
from sklearn import preprocessing
import numpy as np

In [2]:
# Sort

text = open("data\\romeoandjuliet.txt", "r").read()
text = text[:round(len(text)/8)]

chardict = sorted(list(set(text)))

total = len(text)
chars = len(chardict)

print("Total Charaters        :", total)
print("Total Unique Charaters :", chars)

Total Charaters        : 6572
Total Unique Charaters : 64


In [3]:
# Format

chunklength = 50
step = 1
sentences = []
characters = []

for i in range(0, len(text) - chunklength, step):
    sentences.append(text[i : i + chunklength])
    characters.append(text[i + chunklength])

chunks = len(sentences)
print("Total Chunks :", chunks)

Total Chunks : 6522


In [4]:
# Format

x = np.zeros(chunks * chunklength * chars, np.bool).reshape(chunks, chunklength, chars)
y = np.zeros(chunks * 1 * chars, np.bool).reshape(chunks, chars)

for i,v in enumerate(sentences):
    for a,b in enumerate(v):
        x[i][a][chardict.index(b)] = True

for i,v in enumerate(characters):
    y[i][chardict.index(v)] = True

print("Total Data Values  : ", chunks * chunklength * chars)
print("Total Label Values : ", chunks * chars)
print("X Shape :", x.shape)
print("Y Shape :", y.shape)

Total Data Values  :  20870400
Total Label Values :  417408
X Shape : (6522, 50, 64)
Y Shape : (6522, 64)


In [5]:
# Model

model = Sequential()
model.add(LSTM(chars, return_sequences=True, input_shape=(chunklength, chars)))
model.add(Dense(chars))
model.add(Flatten())
model.add(Dense(chars, dtype="float64"))
model.add(Activation("softmax", dtype="float64"))

model.summary()

model.compile(optimizer="rmsprop", loss=categorical_crossentropy)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 50, 64)            33024     
_________________________________________________________________
dense (Dense)                (None, 50, 64)            4160      
_________________________________________________________________
flatten (Flatten)            (None, 3200)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                204864    
_________________________________________________________________
activation (Activation)      (None, 64)                0         
Total params: 242,048
Trainable params: 242,048
Non-trainable params: 0
_________________________________________________________________


In [6]:
# Train

model.fit(x=x, y=y, batch_size=chunklength, epochs=60)

Epoch 1/60
131/131 [==============================] - 7s 52ms/step - loss: 3.3308
Epoch 2/60
131/131 [==============================] - 7s 55ms/step - loss: 3.1056
Epoch 3/60
131/131 [==============================] - 7s 55ms/step - loss: 2.7310
Epoch 4/60
131/131 [==============================] - 7s 55ms/step - loss: 2.3961
Epoch 5/60
131/131 [==============================] - 7s 55ms/step - loss: 2.0983
Epoch 6/60
131/131 [==============================] - 7s 56ms/step - loss: 1.8605
Epoch 7/60
131/131 [==============================] - 7s 55ms/step - loss: 1.6623
Epoch 8/60
131/131 [==============================] - 7s 55ms/step - loss: 1.4962
Epoch 9/60
131/131 [==============================] - 7s 55ms/step - loss: 1.3442
Epoch 10/60
131/131 [==============================] - 7s 55ms/step - loss: 1.2098
Epoch 11/60
131/131 [==============================] - 7s 55ms/step - loss: 1.0858
Epoch 12/60
131/131 [==============================] - 7s 55ms/step - loss: 0.9801
Epoch 13/60
1

In [8]:
# User Input

userinput = chardict[0] * 5 or sentences[0]

userdata = np.zeros(chunklength * chars, np.bool).reshape(1, chunklength, chars)

for i,v in enumerate(userinput):
        userdata[0][i][chardict.index(v)] = True

In [9]:
# Input Data

inputdata = x[1].reshape(1, chunklength, chars)

print("Input shape : ", inputdata.shape)

Input shape :  (1, 50, 64)


In [14]:
print(x[1])

[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False  True False ... False False False]
 [False False False ... False False False]]


In [12]:
# Sample

totalclean = ""

for i in range(chunklength * 5):
    prediction = model.predict(inputdata).astype("float64")
    prediction = np.random.multinomial(1, prediction[0], 1)

    for a,b in enumerate(prediction[0]):
        if b == 1:
            prediction[0][a] = True            
            totalclean += chardict[a]
        else:
            prediction[0][a] = False

    inputdata[0] = np.append(inputdata[0][1:], prediction).reshape(chunklength, chars)

In [13]:
print(totalclean)

ung Nobleman, kinsman to the Prince.
Montague,}Heads of two Houses at variance with each other.
Capulet, }
An Old Man, Uncle to Capulet.
Romeo, Son to Montague.
Mercutio, Kinsman to the Prince, and Friend to Romeo.
Benvolio, Nephew to Montague, and F


In [7]:
model.save("models/novelbot2-1")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: models/novelbot2-1\assets
